In [5]:
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import tifffile as tf

In [4]:
path_dataset = Path('/home/CSHL2024_files/Team_3P3P/3P3P Day3/2P_streaming_vis_stim_500um')

In [8]:
dot2p_folder = path_dataset / (path_dataset.stem + '.2p')

In [9]:
path_img = next(dot2p_folder.glob('*.tif'))

In [26]:
img = tf.imread(path_img)[:, :, 1]
fig = plt.figure(figsize=(15,7))
ax = fig.subplots(1,2)
ax[0].imshow(img, cmap='gray')
ax[1].hist(np.ravel(img), np.arange(0, 600, 10))




Error in callback <function _draw_all_if_interactive at 0x702bfc096a60> (for post_execute), with arguments args (),kwargs {}:


AttributeError: 'str' object has no attribute 'get_position'

AttributeError: 'str' object has no attribute 'get_position'

<Figure size 1500x700 with 2 Axes>